### In-Medium SRG in the TensorNetwork architecture
This code implements the IM-SRG flow equations in the TensorNetwork architecture. TensorNetwork is an open source library that optimizes tensor network algorithms, using TensorFlow in the backend. The library provides an intuitive interface for writing tensor networks in a graphical representation. See <a href="https://arxiv.org/pdf/1905.01330.pdf">arXiv:1905.01330</a> for an introduction to the library.

TensorNetwork library --> <a href="https://www.github.com/google/TensorNetwork">TensorNetwork Github</a> <br>
TensorFlow library (this code uses core release 1.13.1)--> <a href="https://www.tensorflow.org">TensorFlow website</a> <br>

In [1]:
import numpy as np
import tensorflow as tf
tf.enable_v2_behavior()
import tensornetwork
print(tf.__version__)

1.13.1


In [20]:
# --- BUILD HAMILTONIAN -----------
def build_hamiltonian(n_hole_states, n_particle_states):
    numh = n_hole_states
    nump = n_particle_states
    nums = numh + nump
    
    ref = np.append(np.ones(numh), np.zeros(nump))
    holes = np.arange(numh)
    particles = np.arange(numh,numh+nump)
    B1 = np.append(holes,particles)
    
    # one body part of Hamiltonian is floor-division of basis index
    # matrix elements are (P-1) where P is energy level
    H1B = np.diag(np.floor_divide(B1,2))

    H2B = np.zeros((nums, nums, nums, nums))
    for p in B1:
        for q in B1:
            for r in B1:
                for s in B1:

                    pp = np.floor_divide(p,2)
                    qp = np.floor_divide(q,2)
                    rp = np.floor_divide(r,2)
                    sp = np.floor_divide(s,2)

                    ps = 1 if p%2==0 else -1
                    qs = 1 if q%2==0 else -1
                    rs = 1 if r%2==0 else -1
                    ss = 1 if s%2==0 else -1

                    if pp != qp or rp != sp:
                        continue
                    if ps == qs or rs == ss:
                        continue
                    if ps == rs and qs == ss:
                        H2B[p,q,r,s] = -0.25
                    if ps == ss and qs == rs:
                        H2B[p,q,r,s] = 0.25
                        
    return (H1B, H2B, ref, holes, particles, B1)

# covers na - nb
def get_occA(B1_basis, ref):
    n = len(B1_basis)
    occA = np.zeros((n,n,n,n))
    
    for a in B1_basis:
        for b in B1_basis:
            occA[a,b,a,b] = ref[a] - ref[b]
            
    return occA
        
# covers (1-na-nb)
def get_occB(B1_basis, ref):
    n = len(B1_basis)    
    occB = np.zeros((n,n,n,n))
    
    for a in B1_basis:
        for b in B1_basis:
            occB[a,b,a,b] = 1 - ref[a] - ref[b]
            
    return occB
        
# covers na*nb + (1-na-nb)*nc
def get_occC(B1_basis, ref):
    n = len(B1_basis)        
    occC = np.zeros((n,n,n,n,n,n))
    
    for a in B1_basis:
        for b in B1_basis:
            for c in B1_basis:
                occC[a,b,c,a,b,c] = ref[a]*ref[b] + (1-ref[a]-ref[b])*ref[c]
                
    return occC

# covers na*nb*(1-nc-nd) + na*nb*nc*nd
def get_occD(B1_basis, ref):
    n = len(B1_basis)    
    occD = np.zeros((n,n,n,n))
    
    for a in B1_basis:
        for b in B1_basis:
            for c in B1_basis:
                for d in B1_basis:
                    occD[a,b,c,d] = ref[a]*ref[b]*(1-ref[c]-ref[d])+ref[a]*ref[b]*ref[c]*ref[d]
                    
    return occD

def create_network(node_list):
    net = tensornetwork.TensorNetwork()
    assert isinstance(node_list, list), "argument must be a list of nodes"
    
    node_instances = []
    for new_node in node_list:
        node = net.add_node(new_node)
        node_instances.append(node)
        
    return (net, node_instances)
                

In [21]:
# --- NORMAL ORDER HAMILTONIAN -----------
# Calculate 0b, 1b, 2b pieces 
#
# zero-body piece is scalar
# one-body piece is rank 2 tensor
# two-body piece is rank 4 tensor

def normal_order(H1B_t, H2B_t, holes, particles):
    bas1B = np.append(holes,particles)
    
    net = tensornetwork.TensorNetwork()
    
    # - Calculate 0B piece
    H1B_holes = H1B_t[np.ix_(holes,holes)]
    H2B_holes = H2B_t[np.ix_(holes,holes,holes,holes)]
    
    ob_node0b = net.add_node(H1B_holes)
    tb_node0b = net.add_node(H2B_holes)
    
    ob_ii = net.connect(ob_node0b[0],ob_node0b[1])
    tb_ijij1 = net.connect(tb_node0b[0], tb_node0b[2])
    tb_ijij2 = net.connect(tb_node0b[1], tb_node0b[3])
    
    flatten = net.flatten_edges([tb_ijij1, tb_ijij2])
    ob_contract = net.contract(ob_ii).tensor.numpy()
    tb_contract = 0.5*net.contract(flatten).tensor.numpy()

    E = ob_contract + tb_contract
    
    
    # - Calculate 1B piece
    ob_node1b = net.add_node(H1B_t)
    tb_node1b = net.add_node(H2B_t[np.ix_(bas1B,holes,bas1B,holes)])
    
    tb_ihjh = net.connect(tb_node1b[1], tb_node1b[3])
    tb_contract = net.contract(tb_ihjh)
    
    f = ob_node1b.tensor.numpy() + tb_contract.tensor.numpy()
    
    G = H2B_t
    
    return (E, f, G)

In [31]:
def wegner(f, G, holes, particles, occA, occB, occC, occD):
    bas1B = np.append(holes,particles)
    
#     net = tensornetwork.TensorNetwork()
#     occA_node = net.add_node(occA)
#     occB_node = net.add_node(occB)
#     occC_node = net.add_node(occC)
#     occD_node = net.add_node(occD)
    
    # - Decouple off-diagonal 1B and 2B pieces
    fod = np.zeros(f.shape)
    fod[np.ix_(particles, holes)] += f[np.ix_(particles, holes)]
    fod[np.ix_(holes, particles)] += f[np.ix_(holes, particles)]
    fd = f - fod
    
    God = np.zeros(G.shape)
    God[np.ix_(particles, particles, holes, holes)] += G[np.ix_(particles, particles, holes, holes)]
    God[np.ix_(holes, holes, particles, particles)] += G[np.ix_(holes, holes, particles, particles)]
    Gd = G - God
    
#     fd_node = net.add_node(fd)
#     Gd_node = net.add_node(Gd)
#     fod_node = net.add_node(fod)
#     God_node = net.add_node(God)
    
    # - Calculate 1B piece of generator
    net_1b1, nodes = create_network([fd, Gd, fod, God])
    fd_node = nodes[0]
    Gd_node = nodes[1]
    fod_node = nodes[2]
    God_node = nodes[3]
#     print(type(fd_node))
    
    sum1_edge = net_1b1.connect(fd_node[1], fod_node[0], name="eta1b_fdfod_edge")
    sum1_cont = net_1b1.contract(sum1_edge, name="eta1b_fdfod_contract")
    sum1 = sum1_cont.tensor.numpy() - np.transpose(sum1_cont.tensor.numpy())
    
    net_1b2, nodes = create_network([fd, Gd, fod, God, occA])
    fd_node = nodes[0]
    Gd_node = nodes[1]
    fod_node = nodes[2]
    God_node = nodes[3]
    occA_node = nodes[4]
    
    sum2_edgeA_fdGod = net_1b2.connect(fd_node[0], God_node[2], name="eta1B_fdGod_edgeA")
    sum2_edgeB_fdGod = net_1b2.connect(fd_node[1], God_node[0], name="eta1B_fdGod_edgeB")
    sum2_edgeA_fodGd = net_1b2.connect(fod_node[0], Gd_node[2], name="eta1B_fodGd_edgeA")
    sum2_edgeB_fodGd = net_1b2.connect(fod_node[1], Gd_node[0], name="eta1B_fodGd_edgeB")
    sum2_edge_fdGod_f = net_1b2.flatten_edges([sum2_edgeA_fdGod, sum2_edgeB_fdGod])
    sum2_edge_fodGd_f = net_1b2.flatten_edges([sum2_edgeA_fodGd, sum2_edgeB_fodGd])
    sum2_cont_fdGod = net_1b2.contract(sum2_edge_fdGod_f)
    sum2_cont_fodGd = net_1b2.contract(sum2_edge_fodGd_f)
    sum2_edgeA_occA = net_1b2.connect()
    
    return sum1_cont

In [32]:
H1B, H2B, ref, holes, particles, B1 = build_hamiltonian(4,4)
E, f, G = normal_order(H1B, H2B, holes, particles)

occA = get_occA(B1, ref)
occB = get_occB(B1, ref)
occC = get_occC(B1, ref)
occD = get_occD(B1, ref)

In [33]:
test = wegner(f, G, holes, particles, occA, occB, occC, occD)
print(type(test))

<class 'tensornetwork.network_components.Node'>
